# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier

from functions import train_model

In [2]:
# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

## Data understanding



**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)


** -> Data is cleaned and ready to be used**

In [ ]:
portfolio

In [ ]:
portfolio.info()

In [ ]:
#portfolio['channels'].explode() #.unique()

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**-> the NoneType gender will be removed from profile dataset and storage in a new dataset**

In [ ]:
profile.head()

In [ ]:
profile.info()

In [ ]:
profile.describe(include='all')

In [ ]:
profile['gender'].value_counts(dropna=False)

In [ ]:
profile[profile['gender'].isna()]['age'].value_counts(dropna=False)

In [ ]:
profile[profile['gender'].isna()]['income'].value_counts(dropna=False)

In [ ]:
profile.info()

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [ ]:
transcript.head()

In [ ]:
transcript['time'].max()

In [ ]:
transcript.info()

In [ ]:
transcript['value']

In [ ]:
transcript['event'].unique()

## Adjusting the data, column names and details 
**Standardizing column names and simplifying identifiers**

**The keys in 'value' dicts will be changed replacing ' ' by '_'**

In [4]:
profile_none = profile[profile['gender'].isna()].copy()
profile = profile[~profile['gender'].isna()].copy()

# change the id GUIDs (Globally Unique Identifiers) to short tag.
port_id = {
    'ae264e3637204a6fb9bb56bc8210ddfd': 'ofr_A',
    '4d5c57ea9a6940dd891ad53e9dbe8da0': 'ofr_B',
    '3f207df678b143eea3cee63160fa8bed': 'ofr_C',
    '9b98b8c7a33c4b65b9aebfe6a799e6d9': 'ofr_D',
    '0b1e1539f2cc45b7b9fa7c272da2e1d7': 'ofr_E',
    '2298d6c36e964ae4a3e7e9706d1fb8c2': 'ofr_F',
    'fafdcd668e3743c1bb461111dcafc2a4': 'ofr_G',
    '5a8bc65990b245e5a138643cd4eb9837': 'ofr_H',
    'f19421c1d4aa40978ebb69ca19b0e20d': 'ofr_I',
    '2906b810c7d4411798c6938adc9daaa5': 'ofr_J'
}

portfolio['id_short'] = portfolio['id'].map(port_id)

# Column renaming
transcript['event'] = transcript['event'].str.replace(' ','_')

# Fixing the dicts key names
def fix_offer_id(value):
    if isinstance(value, dict) and 'offer id' in value:
        value['offer_id'] = value.pop('offer id')
    return value

transcript['value'] = transcript['value'].apply(fix_offer_id)

In [5]:
set([list(x.keys())[0] for x in transcript['value']])

{'amount', 'offer_id'}

In [6]:
# creating a dataframe with all the columns in the transcript json handling the data in 'value' column
value_df = pd.json_normalize(transcript['value']).fillna(0)
value_df['offer_id'] = value_df['offer_id'].map(port_id).fillna('tran')
transcript_b = pd.concat([transcript, value_df], axis=1).drop('value', axis=1)

In [7]:
transcript_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   person    306534 non-null  object 
 1   event     306534 non-null  object 
 2   time      306534 non-null  int64  
 3   offer_id  306534 non-null  object 
 4   amount    306534 non-null  float64
 5   reward    306534 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 14.0+ MB


# Data Transforming

### Creating the `person_activity` Table with Focus on `customer_id` interactions

The goal is to create a table called `person_activity` that captures the behavior of customers by tracking their activities throughout the event timestamp values.

#### **Objective:**
Track and organize customer interactions (`person`, `event`, `offer_id`) into a structured `timeline` and generate relevant time-based features.

---

#### **Strategy:**

1. **Add a `tag` column** to the `person_activity` table:
   - Enumerate each unique customer activity where the combination of `['person', 'event', 'offer_id']` represents a distinct event.
   
2. **Use a pivot table**:
   - Create a table that provides a unique timeline of customer interactions with offers to which they were exposed. The `tag` column will serve as the index for this operation, since the `offer_id` is not unique across the same `person` and `event`.

3. **Generate features**:
   - Derive features based on the timeline of the **offer exposed** events.

---

#### **Time-Based Features:**

Five key time-based features will be created:

1. **`ofr_influence`**:
   - `True` if the `offer_completed` timestamp is equal or later than the `offer_viewed` timestamp. `False` otherwise.

2. **`extrm_ofr_influence`**:
   - `True` if the `offer_completed` timestamp is exactly to the `offer_received` timestamp. `False` otherwise.

3. **`fidelity_vr`**:
   - The time difference between `offer_viewed` and `offer_received`: Indicates how quickly the customer viewed the offer after receiving it.

4. **`fidelity_cv`**:
   - The time difference between `offer_completed` and `offer_viewed`: Indicates how quickly the customer completed the offer after viewing it.

5. **`overal_cr`**:
   - The time difference between `offer_completed` and `offer_received`: Represents the overall time taken by the customer to complete the offer after receiving it.

---

#### **Result:**

The resulting `person_activity` table will contain:

- **Each `customer_id`** linked to their respective `offer_id` which they were exposed.
- A timeline of events for each offer the customer was exposed to, recorded in columns:
  - `offer_completed`
  - `offer_received`
  - `offer_viewed`

- A set of features derived from these events:
   - `ofr_influence`
   - `extrm_ofr_influence`
   - `fidelity_vr`
   - `fidelity_cv`
   - `overal_cr`

This structured format enables better tracking and analysis of customer behaviors

   - **Insights:** How customers interact with offers over time.

In [54]:
transcript_b['tag'] = (
    transcript_b.groupby(['person', 'event', 'offer_id'])
    .cumcount()
)


person_activity = transcript_b[transcript_b['event'] != 'transaction'].pivot_table(
    index=['person', 'offer_id', 'tag'], 
    columns='event',
    values='time', 
    margins=False, 
    dropna=True,
    #fill_value=0
    
        ).reset_index()

transactions = transcript_b[transcript_b['event'] == 'transaction'][['person', 'time', 'amount']]

person_activity = pd.merge(
    how='left',
    left=person_activity,
    right=transactions,
    left_on=['person', 'offer_completed'],
    right_on=['person', 'time'],
).drop(columns=['time'])

grouped1 = person_activity.groupby(['person', 'offer_id'])

aggregations1 = grouped1.agg(
    n_rec = ('offer_received', 'count'),
    n_vie = ('offer_viewed', 'count'),
    n_com = ('offer_completed', 'count'),
    
)

person_activity = pd.merge(
    how='left',
    left=person_activity,
    right=aggregations1,
    on=['person', 'offer_id']
)


grouped2 = transcript_b[transcript_b['event'] == 'transaction'].groupby(['person', 'offer_id'])

aggregations2 = grouped2.agg(
    n_tran = ('event', 'count'),
    amt_mean = ('amount', 'mean')

)

person_activity['to_vr'] = person_activity['offer_viewed'] - person_activity['offer_received']
person_activity['to_cv'] = person_activity['offer_completed'] - person_activity['offer_viewed'] 
#person_activity['to_vie'] = 714 - person_activity['offer_viewed']

person_activity['curiosity_vr'] = (1 - person_activity['to_vr'] / (person_activity['offer_received'] + person_activity['to_vr'] + person_activity['to_cv']))
person_activity['eagerness_cv'] = person_activity['offer_completed'] / person_activity['offer_viewed']
person_activity['overall_cr'] = person_activity['offer_completed'] / person_activity['offer_received']
                                     
person_activity['ofr_influence'] = person_activity['offer_completed'] >= person_activity['offer_viewed'] 
person_activity['extrm_ofr_influence'] = person_activity['offer_completed'] == person_activity['offer_viewed']


person_activity = pd.merge(
    how='left',
    left=person_activity,
    right=aggregations2,
    on=['person']
)


person_activity.loc[:,'amount'] = person_activity.loc[:,'amount'].fillna(0)

person_activity = person_activity.round(2)

person_activity

,person,offer_id,tag,offer_completed,offer_received,offer_viewed,amount,n_rec,n_vie,n_com,to_vr,to_cv,curiosity_vr,eagerness_cv,overall_cr,ofr_influence,extrm_ofr_influence,n_tran,amt_mean
0,0009655768c64bdeb2e877511632db8f,ofr_C,0,NaN,336.0,372.0,0.00,1,1,0,36.0,NaN,NaN,NaN,NaN,False,False,8.0,15.95
1,0009655768c64bdeb2e877511632db8f,ofr_G,0,528.0,504.0,540.0,14.11,1,1,1,36.0,-12.0,0.93,0.98,1.05,False,False,8.0,15.95
2,0009655768c64bdeb2e877511632db8f,ofr_H,0,NaN,168.0,192.0,0.00,1,1,0,24.0,NaN,NaN,NaN,NaN,False,False,8.0,15.95
3,0009655768c64bdeb2e877511632db8f,ofr_I,0,414.0,408.0,456.0,8.57,1,1,1,48.0,-42.0,0.88,0.91,1.01,False,False,8.0,15.95
4,0009655768c64bdeb2e877511632db8f,ofr_J,0,576.0,576.0,NaN,10.27,1,0,1,NaN,NaN,NaN,NaN,1.00,False,False,8.0,15.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76272,ffff82501cea40309d5fdd7edcca4a07,ofr_E,0,198.0,168.0,174.0,22.88,1,1,1,6.0,24.0,0.97,1.14,1.18,True,False,15.0,15.07
76273,ffff82501cea40309d5fdd7edcca4a07,ofr_G,0,60.0,0.0,6.0,16.06,1,1,1,6.0,54.0,0.90,10.00,inf,True,False,15.0,15.07
76274,ffff82501cea40309d5fdd7edcca4a07,ofr_J,0,384.0,336.0,354.0,15.57,3,3,3,18.0,30.0,0.95,1.08,1.14,True,False,15.0,15.07
76275,ffff82501cea40309d5fdd7edcca4a07,ofr_J,1,414.0,408.0,414.0,17.55,3,3,3,6.0,0.0,0.99,1.00,1.01,True,True,15.0,15.07


In [55]:
person_activity[person_activity['person'] == 'ffff82501cea40309d5fdd7edcca4a07']

,person,offer_id,tag,offer_completed,offer_received,offer_viewed,amount,n_rec,n_vie,n_com,to_vr,to_cv,curiosity_vr,eagerness_cv,overall_cr,ofr_influence,extrm_ofr_influence,n_tran,amt_mean
76271,ffff82501cea40309d5fdd7edcca4a07,ofr_D,0,504.0,504.0,534.0,7.79,1,1,1,30.0,-30.0,0.94,0.94,1.00,False,False,15.0,15.07
76272,ffff82501cea40309d5fdd7edcca4a07,ofr_E,0,198.0,168.0,174.0,22.88,1,1,1,6.0,24.0,0.97,1.14,1.18,True,False,15.0,15.07
76273,ffff82501cea40309d5fdd7edcca4a07,ofr_G,0,60.0,0.0,6.0,16.06,1,1,1,6.0,54.0,0.90,10.00,inf,True,False,15.0,15.07
76274,ffff82501cea40309d5fdd7edcca4a07,ofr_J,0,384.0,336.0,354.0,15.57,3,3,3,18.0,30.0,0.95,1.08,1.14,True,False,15.0,15.07
76275,ffff82501cea40309d5fdd7edcca4a07,ofr_J,1,414.0,408.0,414.0,17.55,3,3,3,6.0,0.0,0.99,1.00,1.01,True,True,15.0,15.07
76276,ffff82501cea40309d5fdd7edcca4a07,ofr_J,2,576.0,576.0,582.0,14.23,3,3,3,6.0,-6.0,0.99,0.99,1.00,False,False,15.0,15.07


In [ ]:
person_activity.info()

In [ ]:
grouped1 = transcript_b[transcript_b['event'] == 'offer_completed'].groupby(['person', 'offer_id'])
grouped2 = transcript_b[transcript_b['event'] == 'transaction'].groupby(['person', 'offer_id'])

aggregations1 = grouped1.agg(
    offer_fidelity=('event', 'count'),
    first_comp =('time', 'min'), 
    last_comp =('time', 'max'), 
).reset_index()

aggregations2 = grouped2.agg(
    fidelity_tran=('event', 'count'),
    first_time_tr=('time', 'min'),
    last_time_tr=('time', 'max')

).reset_index()

fidelity = pd.merge(
    how='left',
    left=aggregations1,
    right=aggregations2,
    on=['person']
)

fidelity['lst_time_to_end_tr'] = transcript_b['time'].max() - fidelity['last_time_tr']
fidelity['lst_time_to_end_com'] = transcript_b['time'].max() - fidelity['last_comp']

fidelity

In [ ]:
fidelity.info()

In [ ]:
aggregations1

In [ ]:
aggregations2

In [ ]:
person_activity[person_activity['person'] == 'ffff82501cea40309d5fdd7edcca4a07']

In [ ]:
fidelity[fidelity['person'] == 'ffff82501cea40309d5fdd7edcca4a07']

In [ ]:
transcript_b[transcript_b['person'] == 'ffff82501cea40309d5fdd7edcca4a07']

In [ ]:
transcript_transaction = transcript_b[transcript_b['event'] == 'transaction'].copy()

person_activity_tran = pd.merge(
    how='left', left=person_activity,
    right=transcript_transaction,
    left_on=['person', 'offer_completed'], 
    right_on=['person', 'time']
    ).drop(columns=['event', 'time',	'offer_id_y',	'reward',	'tag_y', 'tag_x'])

person_activity_tran['amount'] = person_activity_tran['amount'].fillna(0)

person_activity_tran = pd.merge(
    how='left',
    left=person_activity_tran,
    right=profile,
    left_on='person',
    right_on='id'
).drop(columns=['id'])

person_activity_tran = pd.merge(
    how='left',
    left=person_activity_tran,
    right=fidelity,
    left_on=['person', 'offer_id_x'],
    right_on=['person', 'offer_id_y']
)

#person_activity_tran = person_activity_tran

person_activity_tran

In [ ]:
person_activity_tran.info()

In [ ]:
grouped = person_activity_tran.groupby(['offer_completed', 'offer_id_x_x'])

aggregations = grouped.agg(
    amount=('amount', 'sum'),
)

aggregations.plot()

In [ ]:
aggregations

In [ ]:
person_activity_tran[person_activity_tran['person'] == 'ffff82501cea40309d5fdd7edcca4a07']

In [ ]:
# Realizando o agrupamento uma única vez
grouped = person_activity_tran.groupby('offer_id_x_x')

# Calculando todas as métricas necessárias
aggregations = grouped.agg(
    tot_ofr=('ofr_influence', 'count'),
    tot_amt=('amount', 'sum'),
    avr_amt=('amount', 'mean'),
    influ=('ofr_influence', 'sum'),
    ex_influ=('extrm_ofr_influence', 'sum'),
    ofr_inf_rate=('ofr_influence', 'mean'),
    ex_inf_rate=('extrm_ofr_influence', 'mean')

).reset_index()

# Arredondando os valores para 2 casas decimais
result = aggregations.round(2)

result['ext_prop'] = result['ex_influ']/result['influ']  

offer_summary = pd.merge(
    how='left', 
    left=result,
    right=portfolio,
    left_on=['offer_id_x_x'], 
    right_on=['id_short'],
    
    ).drop(columns=['id','id_short'])


offer_summary = offer_summary.fillna(0)

offer_summary


In [ ]:
grouped = person_activity_tran.groupby(['gender', 'offer_id_x_x'])

# Calculando todas as métricas necessárias
aggregations = grouped.agg(
    ofr_inf_sum=('ofr_influence', 'sum'),
    ex_inf_sum=('extrm_ofr_influence', 'sum'),
    ofr_inf_rate=('ofr_influence', 'mean'),
    ex_inf_rate=('extrm_ofr_influence', 'mean'),
    amt_sum=('amount', 'sum'),
    amt_mean=('amount', 'mean'),
    ofr_inf_cnt=('ofr_influence', 'count')
).reset_index()

# Arredondando os valores para 2 casas decimais
result = aggregations.round(2)

result

In [234]:
def categorize_age(age):
    if age < 25:
        return 'Young Adult'
    elif 25 <= age < 40:
        return 'Adult'
    elif 40 <= age < 60:
        return 'Middle-Aged'
    else:
        return 'Senior'

# Aplicar a função ao DataFrame
tb_transactions['age_segment'] = tb_transactions['age'].apply(categorize_age)

In [38]:
#df = df.dropna()

X = df.drop(columns = ['person', 'event', 'became_member_on', 'gender', 'income'])
y = df['gender']

In [ ]:
y.value_counts()

In [ ]:
model1 = RandomForestClassifier(
    random_state=42,
    max_depth=15,
    n_estimators=25,
    min_samples_leaf=2,
    min_samples_split=2,
    #max_features=0.5,
    n_jobs=-1    
    )

model2 = DecisionTreeClassifier(
    random_state=42,
    criterion='entropy',            # Função de impureza para divisão ('gini' ou 'entropy')
    max_depth=15,                # Profundidade máxima da árvore (controle de overfitting)
    min_samples_split=2,         # Número mínimo de amostras para dividir um nó
    min_samples_leaf=2,          # Número mínimo de amostras em uma folha
    #max_features=5,           # Número máximo de features consideradas em cada divisão
    #splitter='random'            # Estratégia para escolher a divisão ('best' ou 'random')
    )



In [ ]:
model_type = model2
train_model(X, y, model_type)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# Instanciar o modelo
rf = RandomForestClassifier(random_state=42)

# Definir a grade de parâmetros
param_grid = {
    'n_estimators': [50, 100, 200],         # Número de árvores na floresta
    'max_depth': [None, 10, 20, 30],       # Profundidade máxima da árvore
    'min_samples_split': [2, 5, 10],       # Número mínimo de amostras para dividir um nó
    'min_samples_leaf': [1, 2, 4],         # Número mínimo de amostras em uma folha
    'max_features': ['sqrt', 'log2'],      # Número de recursos considerados para divisão
    'bootstrap': [True, False]             # Amostragem com ou sem reposição
}

# Configurar o GridSearchCV
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,                  # Validação cruzada (5-fold)
    scoring='accuracy',    # Métrica de avaliação
    verbose=2,             # Nível de detalhe das mensagens
    n_jobs=-1              # Paralelização (usar todos os núcleos disponíveis)
)

# Ajustar o modelo aos dados de treino
grid_search.fit(X_train, y_train)

# Exibir os melhores parâmetros e a melhor pontuação
print("Melhores parâmetros:", grid_search.best_params_)
print("Melhor pontuação (treino):", grid_search.best_score_)

# Avaliar o modelo com os melhores parâmetros nos dados de teste
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Acurácia nos dados de teste:", test_accuracy)


y_pred_train = best_model.predict(X_train)
y_pred = best_model.predict(X_test)

# 1. Confusion Matrix
cm1 = confusion_matrix(y_train, y_pred_train)
cm2 = confusion_matrix(y_test, y_pred)


# 2. Relatório com Precision, Recall e F1-Score
report1 = classification_report(y_test, y_pred, target_names=best_model.classes_, zero_division=1)
print("Test Classification Report:")
print(report1)

report2 = classification_report(y_train, y_pred_train, target_names=best_model.classes_, zero_division=1)
print("Train Classification Report:")
print(report2)